# Prova 3

Instructions:
- Exercises are grouped by topic and ordered by increasing difficulty within each topic.
- Submit your solutions in a well-organized Jupyter notebook.
- You are responsible for creating robust test cases for each question. Ensure your tests cover all possible cases, especially edge cases (e.g., with the largest possible N ).
- Verify that all your solutions meet the specified time constraints. Unless otherwise stated, each question has a 1-second time limit. Some of the later questions may have different limits, which will be indicated.
- Be sure that the inputs (which will be created by you) match the specifications, and your solutions meets the required time limit.
- Presentation matters: make sure your notebook is clean, well-structured, and that both solutions and test cases are clearly labeled.

In [133]:
# from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import (
    HGate, 
    ZGate, 
    XGate, 
    YGate,
)
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import (
    plot_bloch_multivector,
    array_to_latex, 
    plot_histogram,
)
from sympy.parsing.sympy_parser import parse_expr
from sympy.logic.inference import satisfiable


import numpy as np
from numpy.random import default_rng, shuffle

rng = default_rng()

## Distinguishing Unitaries and States

**Exercise 1.** You are given an operation that implements a single-qubit unitary transformation: either the H gate or the X gate. The operation will have Adjoint and Controlled variants defined. Your task is to perform necessary operations and measurements to figure out which unitary it was and to return 0 if it was the H gate or 1 if it was the X gate. You are allowed to apply the given operation and its adjoint/controlled variants at most twice. You have to implement an operation which takes a single-qubit operation as an input and returns an integer.

In [161]:
# 0 -> Gate H, 1 -> Gate X 
def solve1(operator) -> int:
    # We'll apply UXU|0> and measure.
    # When U=X -> XXX|0> = X|0> = |1>
    # When U=H -> HXH|0> = HX(|0>+|1>) = H(|0>+|1>) = |0>
    
    qc = QuantumCircuit(1, 1)           # 1 quibit + 1 bit (output measure)
    qc.append(operator, [0])
    qc.x(0)
    qc.append(operator, [0])
    
    qc.measure(0, 0)
    qc.draw('mpl')
    
    sampler = StatevectorSampler()
    job = sampler.run([qc], shots=1)
    result = job.result()[0].data['c'].get_counts() # A map with the measured val

    return int([k for k in result.keys()][0])
    

In [166]:
# Testing 

tests = [i%2 for i in range(20)]
shuffle(tests)
result_test = {'Accepted':0, 'Wrong Answer':0}
tests_inputs = {'X':0, 'H':0}
for tt in tests:
    gate = XGate() if tt == 1 else HGate()
    tests_inputs[('X' if tt == 1 else 'H')] += 1
    expected = tt
    res = solve1(gate)
    if res == expected: 
        result_test['Accepted'] += 1
    else: 
        result_test['Wrong Answer'] += 1
    
### Showing results
print("The case tests:", tests_inputs)
print(result_test)

The case tests: {'X': 10, 'H': 10}
{'Accepted': 20, 'Wrong Answer': 0}
